In [17]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split


In [3]:
df="/kaggle/input/crash-data-queensland-australia-2001-2022"
new_df=[]
for file in os.listdir(df):
    data=os.path.join(df,file)
    new_df.append(pd.read_csv(data))

In [4]:
concaty=pd.concat(new_df,axis=1)

In [5]:
concaty2=concaty.dropna(how='any')

In [11]:
concaty2

,Crash_Year,Crash_Police_Region,Crash_Severity,Involving_Male_Driver,Involving_Female_Driver,Involving_Young_Driver_16-24,Involving_Senior_Driver_60plus,Involving_Provisional_Driver,Involving_Overseas_Licensed_Driver,Involving_Unlicensed_Driver,...,Involving_Drink_Driving,Involving_Driver_Speed,Involving_Fatigued_Driver,Involving_Defective_Vehicle,Count_Crashes,Count_Fatality,Count_Hospitalised,Count_Medically_Treated,Count_Minor_Injury,Count_All_Casualties
17,2001.0,Brisbane,Fatal,Yes,Yes,Yes,No,Yes,No,No,...,No,No,No,Yes,25.0,0.0,0.0,29.0,0.0,29.0
23,2001.0,Brisbane,Hospitalisation,No,Yes,No,Yes,Yes,No,No,...,Yes,Yes,No,No,7.0,0.0,0.0,11.0,0.0,11.0
28,2001.0,Brisbane,Hospitalisation,No,Yes,Yes,No,Yes,No,Yes,...,No,Yes,No,Yes,3.0,0.0,0.0,0.0,3.0,3.0
29,2001.0,Brisbane,Hospitalisation,No,Yes,Yes,Yes,No,No,No,...,Yes,No,No,No,27.0,0.0,0.0,0.0,30.0,30.0
33,2001.0,Brisbane,Hospitalisation,Yes,No,No,No,No,Yes,No,...,Yes,Yes,No,Yes,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,2003.0,North Coast,Property damage only,Yes,No,Yes,Yes,No,No,No,...,No,Yes,No,No,10.0,0.0,12.0,2.0,2.0,16.0
3125,2003.0,Northern,Hospitalisation,No,Yes,Yes,Yes,Yes,No,No,...,No,No,No,No,317.0,0.0,0.0,0.0,366.0,366.0
3201,2003.0,Northern,Minor injury,Yes,Yes,Yes,Yes,No,No,No,...,Yes,No,No,Yes,1.0,0.0,4.0,0.0,0.0,4.0
3203,2003.0,Northern,Property damage only,No,No,No,No,No,No,No,...,Yes,Yes,No,No,5.0,0.0,5.0,0.0,0.0,5.0


In [9]:
obj_col=concaty2.select_dtypes(include=["object","category"]).columns
concaty21=pd.get_dummies(concaty2,columns=obj_col,drop_first=False).astype(int)

In [12]:
concaty21.iloc[:,0:15]

,Crash_Year,Count_Crashes,Count_Casualty_Fatality,Count_Casualty_Hospitalised,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_All,Crash_Year,Casualty_Count,Crash_Ref_Number,Crash_Year,Crash_Hour,Crash_Longitude,Crash_Latitude,Crash_DCA_Code
17,2001,1,1,2,0,0,3,2007,2,18,2001,8,153,-27,301
23,2001,1,0,1,0,0,1,2008,153,24,2001,12,153,-27,303
28,2001,1,0,2,0,0,2,2008,69,29,2001,21,153,-27,309
29,2001,1,0,2,0,0,2,2008,132,30,2001,11,153,-27,202
33,2001,4,0,4,0,0,4,2008,10,34,2001,17,153,-27,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,2003,12,0,0,0,0,0,2004,1,3088,2001,12,153,-27,309
3125,2003,1,0,1,1,0,2,2003,1,3126,2001,20,153,-27,101
3201,2003,2,0,0,0,2,2,2003,1,3202,2002,18,153,-27,704
3203,2003,9,0,0,0,0,0,2005,1,3204,2002,11,153,-27,308


In [13]:
X=concaty21.drop(columns=["Count_Casualty_Fatality","Count_Casualty_Hospitalised","Count_Casualty_MedicallyTreated","Count_Casualty_MinorInjury"])
Y=concaty21[["Count_Casualty_Fatality","Count_Casualty_Hospitalised","Count_Casualty_MedicallyTreated","Count_Casualty_MinorInjury"]]

In [18]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [21]:
X_train.shape

(372, 596)

# # Deep learning

In [72]:
from keras.layers import Dense,LSTM,SimpleRNN,BatchNormalization,Dropout,Input,TimeDistributed,LeakyReLU,Attention,MultiHeadAttention
from keras.models import Model 
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [53]:
early_stopping=EarlyStopping(
monitor='val_loss',
verbose=1,
patience=5)

In [54]:
X_train_reshape=X_train.values 
X_train_reshaped=np.repeat(X_train_reshape[:,np.newaxis,:],12,axis=1)
X_train_reshaped.shape

(372, 12, 596)

In [55]:
Y_train_reshape=Y_train.values 
Y_train_reshaped=np.repeat(Y_train_reshape[:,np.newaxis,:],12,axis=1)
Y_train_reshaped.shape

(372, 12, 12)

In [56]:
Y_test_reshape=Y_test.values 
Y_test_reshaped=np.repeat(Y_test_reshape[:,np.newaxis,:],12,axis=1)
Y_test_reshaped.shape

(93, 12, 12)

In [57]:
Y_train.nunique()

Count_Casualty_Fatality            12
Count_Casualty_Fatality             2
Count_Casualty_Fatality            30
Count_Casualty_Hospitalised        30
Count_Casualty_Hospitalised         5
Count_Casualty_Hospitalised        72
Count_Casualty_MedicallyTreated    42
Count_Casualty_MedicallyTreated     5
Count_Casualty_MedicallyTreated    59
Count_Casualty_MinorInjury         25
Count_Casualty_MinorInjury          4
Count_Casualty_MinorInjury         62
dtype: int64

In [74]:
inputs=Input(shape=(12,596))
s=SimpleRNN(32,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
b=BatchNormalization()(s)
d=Dropout(0.5)(b)
s=SimpleRNN(32,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
b=BatchNormalization()(s)
d=Dropout(0.5)(b)
s=SimpleRNN(64,activation=None,return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
r=LeakyReLU(0.8)(s)
b=BatchNormalization()(r)
d=Dropout(0.5)(b)
s=SimpleRNN(64,activation=None,return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
r=LeakyReLU(0.8)(s)

a=MultiHeadAttention(num_heads=4,key_dim=64)(r,r)


b=BatchNormalization()(a)
s=SimpleRNN(320,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(b)
b=BatchNormalization()(s)
d=Dropout(0.5)(b)
s=SimpleRNN(64,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
b=BatchNormalization()(s)
d=Dropout(0.5)(b)
s=SimpleRNN(640,activation=None,return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
r=LeakyReLU(0.8)(s)
b=BatchNormalization()(r)
d=Dropout(0.5)(b)
s=SimpleRNN(32,activation=None,return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
r=LeakyReLU(0.8)(s)

b=BatchNormalization()(r)
# d=Dropout(0.5)(b)
outputs=TimeDistributed(Dense(12,activation="softmax"))(b)
model=Model(inputs,outputs)
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, 12, 596)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_68       │ (None, 12, 32)    │     20,128 │ input_layer_16[0… │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 32)    │        128 │ simple_rnn_68[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_35          │ (None, 12, 32)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_69       │ (None, 12, 32)    │      2,080 │ dropout_35[0][0]  │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 32)    │        128 │ simple_rnn_69[0]… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 12, 32)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_70       │ (None, 12, 64)    │      6,208 │ dropout_36[0][0]  │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_34      │ (None, 12, 64)    │          0 │ simple_rnn_70[0]… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 64)    │        256 │ leaky_re_lu_34[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_37          │ (None, 12, 64)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_71       │ (None, 12, 64)    │      8,256 │ dropout_37[0][0]  │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_35      │ (None, 12, 64)    │          0 │ simple_rnn_71[0]… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 64)    │     66,368 │ leaky_re_lu_35[0… │
│ (MultiHeadAttentio… │                   │            │ leaky_re_lu_35[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 64)    │        256 │ multi_head_atten… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_72       │ (None, 12, 320)   │    123,200 │ batch_normalizat… │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 320)   │      1,280 │ simple_rnn_72[0]

 Total params: 729,004 (2.78 MB)

 Trainable params: 726,508 (2.77 MB)

 Non-trainable params: 2,496 (9.75 KB)

In [75]:
model.fit(X_train_reshaped,Y_train_reshaped,epochs=32,batch_size=32,validation_data=(X_test_reshaped,Y_test_reshaped),callbacks=[early_stopping])

Epoch 1/32
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 234ms/step - accuracy: 0.0913 - loss: 324.8056 - val_accuracy: 0.0986 - val_loss: 1182.6726
Epoch 2/32
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.0916 - loss: 274.8382 - val_accuracy: 0.0582 - val_loss: 1262.7162
Epoch 3/32
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.1112 - loss: 273.7416 - val_accuracy: 0.1398 - val_loss: 1075.3433
Epoch 4/32
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.1438 - loss: 239.8842 - val_accuracy: 0.1541 - val_loss: 960.0724
Epoch 5/32
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.1828 - loss: 266.8332 - val_accuracy: 0.1443 - val_loss: 763.1637
Epoch 5: early stopping


In [76]:
loss,accuracy=model.evaluate(X_test_reshaped,Y_test_reshaped)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1343 - loss: 921.5537 
